In [9]:
!ls ../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database

audio_and_txt_files      filename_format.txt
filename_differences.txt patient_diagnosis.csv


In [23]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split

In [6]:
demo = pd.read_csv('../input/respiratory-sound-database/demographic_info.txt', 
                   sep=' ', names=['pId', 'age', 'sex', 'adult_bmi', 'child_wt', 'child_ht'])
demo.head()

,pId,age,sex,adult_bmi,child_wt,child_ht
0,101,3.00,F,NaN,19.0,99.0
1,102,0.75,F,NaN,9.8,73.0
2,103,70.00,F,33.00,NaN,NaN
3,104,70.00,F,28.47,NaN,NaN
4,105,7.00,F,NaN,32.0,135.0


In [7]:
demo.drop(['adult_bmi', 'child_wt', 'child_ht'], axis=1, inplace=True)
demo.head()

,pId,age,sex
0,101,3.00,F
1,102,0.75,F
2,103,70.00,F
3,104,70.00,F
4,105,7.00,F


In [10]:
demo.isnull().sum()

pId    0
age    1
sex    1
dtype: int64

In [26]:
demo['age'].fillna(np.mean(demo['age']), inplace=True)
demo['sex'].fillna(demo['sex'].mode, inplace=True)

In [27]:
diagnosis = pd.read_csv('../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv',
                       names=['pId', 'diagnosis'])
diagnosis.head()

,pId,diagnosis
0,101,URTI
1,102,Healthy
2,103,Asthma
3,104,COPD
4,105,URTI


In [28]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pId     126 non-null    int64  
 1   age     126 non-null    float64
 2   sex     126 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 3.1+ KB


In [30]:
diagnosis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pId        126 non-null    int64 
 1   diagnosis  126 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.1+ KB


In [31]:
patients = pd.merge(demo, diagnosis, on='pId')
patients.head()

,pId,age,sex,diagnosis
0,101,3.00,F,URTI
1,102,0.75,F,Healthy
2,103,70.00,F,Asthma
3,104,70.00,F,COPD
4,105,7.00,F,URTI


In [20]:
model = load_model('vgg16_base_res.h5')

In [32]:
pids = patients['pId'].unique()
pids

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204,
       205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217,
       218, 219, 220, 221, 222, 223, 224, 225, 226])

In [ ]:
base_images_loc = '../output/train_test_split'

for pid in pids:
    diag = patients[patients['pId'] == pid]['diagnosis']
    